In [1]:
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error

In [2]:
def reset_graph():
    tf.reset_default_graph()
    
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # there is no regularization loss term
    return max_norm

# 1. Input Data

In [3]:
file_name = "BRI641_fMRI_data_new_sets.mat"
m = loadmat(file_name)
print(m)
# data exploration
# I decide to Use"data_1d_x" for data analysis

{'data_1d_x': array([[-0.03519887, -0.19054532, -0.7751938 , ...,  0.82167444,
        -0.15327348,  0.70570284],
       [-0.50451719, -0.55348879, -0.7751938 , ..., -0.06662225,
        -0.15327348,  0.32424185],
       [-0.03519887, -0.19054532, -0.36175711, ..., -0.06662225,
        -0.15327348, -0.05721915],
       ..., 
       [-0.19350239, -0.34979951, -0.46656299, ..., -0.02357934,
        -0.86529217, -1.40392241],
       [-0.19350239,  0.33273611,  0.26530052, ...,  0.29081192,
        -0.86529217, -0.54656521],
       [-1.00824931, -1.03233512, -0.83249474, ...,  0.29081192,
        -0.86529217, -1.40392241]]), 'data_3d_x': array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,

In [4]:
X_data = m["data_1d_x"] # Input data 
Y_data = m["data_y"]    # labels X_data.shape
print("Input data shape:", X_data.shape ,"/////" ,"Data label shape: ", Y_data.shape)

Input data shape: (1440, 74484) ///// Data label shape:  (1440, 1)


# 2. Feed-Forward Deep Neural Network

### - DNN Architecture
##### Input --> Hidden layer 1 --> Hidden layer 2 --> Output layer

1. There are 2 Hidden layer
2. Using Dropout and Max-Norm Regularization
3. Activation function : relu
4. Optimizer : Gradient Descent Optimizer

In [5]:
reset_graph()

In [6]:
# Hyperparameter

n_inputs = 74484     # input data variables node
n_hidden1 = 300      # hidden layer 1 node
n_hidden2 = 100      # hidden layer 2 node
n_outputs = 4        # output node: output class is 4 (LH / RH / AD / VS)

learning_rate = 0.01 # dropout rate
dropout_rate = 0.5   # dropout rate


n_epochs = 20        # epoch
batch_size = 30      # batch size

In [7]:
# input, output placeholder setting
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [8]:
# Dropout
training = tf.placeholder_with_default(False, shape=(), name='training')
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

# Max-Norm Regularization
max_norm_reg = max_norm_regularizer(threshold=1.0)

# Deep neural network design
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, name="hidden1",
                              activation=tf.nn.relu, kernel_regularizer=max_norm_reg)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, name="hidden2",
                              activation=tf.nn.relu, kernel_regularizer=max_norm_reg)
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

#### I use "tf.layers.dense" function.
#### "tf.layers.dense" is functional interface for the densely-connected layer.
#### In this code, i didn't assign weight. because "tf.layers.dense" parameter has kernel_initializer. and this is Initialize function for the weight matrix. If None (default), weights are initialized using the default initializer used by "tf.get_variable".
#### Default "tf.get_variable" gives weight from uniform distribution. and then it is trained in the network

In [9]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    # Computes sparse softmax cross entropy between logits and labels
    loss = tf.reduce_mean(xentropy, name="loss")
    # Computes the mean of elements across "xentropy"

In [10]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # I implement Gradient Descent Optimizer
    training_op = optimizer.minimize(loss)
    # minimize loss to train

In [11]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Computes accuracy

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# run the weights clipping operations after each training operation
clip_all_weights = tf.get_collection("max_norm")

# Leave one out cross validation - group making
groups = []
for i in range(1,13):
    group = [i] * 120
    for i in group:
        groups.append(i)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_data, Y_data, groups)

looop = []
times = 1
for train_index, test_index in logo.split(X_data, Y_data, groups):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = Y_data[train_index], Y_data[test_index]
    # LOOCV by indexing
        
    with tf.Session() as sess:
        init.run()
        accuracy_test = []
        for epoch in range(n_epochs):
            # Slicing train data by batch size(30)
            i = 0
            for batch in range(len(X_train) // batch_size):
                X_batch = X_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                i += batch_size
                # Training the weights, biases - Deep Neural Networks
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1)})
            # Computes accuracy training set
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1)})
            # Computes accuracy test set
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1)})
            # append the test set accuracy in epoch 
            accuracy_test.append(acc_test)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        # append the biggets test set accuracy in each epoch 
        looop.append(max(accuracy_test))
        save_path = saver.save(sess, "./my_model_final.ckpt")
    print(times," times cross validation End")
    times += 1
    print("*"*30)
print("="*70)
print("Final DNN test accuracy result each cross validation")
print(looop)
print("LOOCV Test accuracy mean in DNN: ", np.mean(looop))

0 Train accuracy: 0.818939 Test accuracy: 0.591667
1 Train accuracy: 0.923485 Test accuracy: 0.816667
2 Train accuracy: 0.939394 Test accuracy: 0.858333
3 Train accuracy: 0.970455 Test accuracy: 0.866667
4 Train accuracy: 0.963636 Test accuracy: 0.85
5 Train accuracy: 0.981818 Test accuracy: 0.891667
6 Train accuracy: 0.993182 Test accuracy: 0.9
7 Train accuracy: 0.996212 Test accuracy: 0.9
8 Train accuracy: 1.0 Test accuracy: 0.891667
9 Train accuracy: 1.0 Test accuracy: 0.891667
10 Train accuracy: 1.0 Test accuracy: 0.891667
11 Train accuracy: 1.0 Test accuracy: 0.883333
12 Train accuracy: 1.0 Test accuracy: 0.891667
13 Train accuracy: 1.0 Test accuracy: 0.891667
14 Train accuracy: 1.0 Test accuracy: 0.891667
15 Train accuracy: 1.0 Test accuracy: 0.891667
16 Train accuracy: 1.0 Test accuracy: 0.891667
17 Train accuracy: 1.0 Test accuracy: 0.891667
18 Train accuracy: 1.0 Test accuracy: 0.891667
19 Train accuracy: 1.0 Test accuracy: 0.891667
1  times cross validation End
**************

### Feed-Forward Deep Neural Network Results Summary

1. Use the leave-one-subject-out cross-validation (LOOCV) for classification test
2. Train set accuracy check, not underfitting or overfitting
3. Extract the biggest accuracy each LOOCV, and then compute average the each accuracy
4. Final test set accuracy is 0.938194 // error rate = 1 - 0.936111 = 0.061806
5. It means 120 smaple classification is correct about 113 sample

# 3. Convolutional Neural Network

### - CNN Architecture
##### Input --> Convolution layer --> Convolution layer --> Pooling layer --> Convolution layer --> Pooling layer --> Neural Network --> output layer

1. There are 6 layer, 3 convolution layer and 2 pooling layer
2. 1-Dimension Convolutional Neural Network
3. Using Dropout
4. Activation function: relu
5. Optimizer: Adam Optimizer

In [13]:
reset_graph()

In [14]:
# Data set is 1-dimension, so i use 1-dimension CNN
height = 1
width = 74484
channels = 1
n_inputs = height * width

In [15]:
# First Convolution layer information
conv1_fmaps = 32
conv1_ksize = 10
conv1_stride = 5
conv1_pad = "SAME"

# Second Convolution layer information
conv2_fmaps = 64
conv2_ksize = 10
conv2_stride = 5
conv2_pad = "SAME"

# Third Convolution layer information(+ dropout rate)
conv3_fmaps = 64
conv3_ksize = 10
conv3_stride = 5
conv3_pad = "SAME"
conv3_dropout_rate = 0.5

# In 2nd pooling layer, to make flat layer
pool2_fmaps = conv3_fmaps

# Neural Network layer information(+ dropout rate)
n_fc1 = 128
fc1_dropout_rate = 0.5

# output node: output class is 4 (LH / RH / AD / VS)
n_outputs = 4

In [16]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=(None,74484), name="X")
    # Conv1d function's shape is required [batch, in_width, in_channels]
    # so, reshape the input X like [-1, width, channels]
    X_reshaped = tf.reshape(X, shape=[-1, width, channels])
    y = tf.placeholder(tf.int64, shape=(None), name="y")
    # To use dropout, make this training part
    training = tf.placeholder_with_default(False, shape=[], name='training')

In [17]:
# Convolutional neural network design
# activation function is relu

conv1 = tf.layers.conv1d(X_reshaped,
                         filters=conv1_fmaps,
                         kernel_size=conv1_ksize,
                         strides=conv1_stride,
                         padding=conv1_pad,
                         activation=tf.nn.relu,
                         name="conv1")
    
conv2 = tf.layers.conv1d(conv1,
                         filters=conv2_fmaps,
                         kernel_size=conv2_ksize,
                         strides=conv2_stride,
                         padding=conv2_pad,
                         activation=tf.nn.relu,
                         name="conv2")

with tf.name_scope("pool1"):
    pool1 = tf.layers.max_pooling1d(conv2,
                                    pool_size=2,
                                    strides=2,
                                    padding="VALID")
     
conv3 = tf.layers.conv1d(pool1,
                         filters=conv3_fmaps,
                         kernel_size=conv3_ksize,
                         strides=conv3_stride,
                         padding=conv3_pad,
                         activation=tf.nn.relu,
                         name="conv3")

with tf.name_scope("pool2"):
    pool2 = tf.layers.max_pooling1d(conv3,
                                    pool_size=2,
                                    strides=2,
                                    padding="VALID")
    # pool2 layer form is (?, 149, 64) so, pool2_fmaps(64)*149 to flatten
    pool2_flat = tf.reshape(pool2,
                            shape=[-1, pool2_fmaps*149])
    pool2_flat_drop = tf.layers.dropout(pool2_flat,
                                        conv3_dropout_rate,
                                        training=training)

In [18]:
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool2_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1_drop, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    # Computes sparse softmax cross entropy between logits and labels
    loss = tf.reduce_mean(xentropy)
    # Computes the mean of elements across "xentropy"
    optimizer = tf.train.AdamOptimizer()
    # I implement Adam Optimizer
    training_op = optimizer.minimize(loss)
    # minimize loss to train

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # Computes accuracy

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [19]:
## This is same with Deep Neural Network session part

# Same with DNN part
n_epochs = 20
batch_size = 30

# Leave one out cross validation - group making
groups = []
for i in range(1,13):
    group = [i] * 120
    for i in group:
        groups.append(i)
logo = LeaveOneGroupOut()
logo.get_n_splits(X_data, Y_data, groups)

looop = []
times = 1
for train_index, test_index in logo.split(X_data, Y_data, groups):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = Y_data[train_index], Y_data[test_index]
    with tf.Session() as sess:
        init.run()
        accuracy_test = []
        for epoch in range(n_epochs):
            i = 0
            for batch in range(len(X_train) // batch_size):
                X_batch = X_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                i += batch_size
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1)})
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1)})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1)})
            accuracy_test.append(acc_test)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        looop.append(max(accuracy_test))
        save_path = saver.save(sess, "./my_CNN_model_final.ckpt")
    print(times," times cross validation End")
    times += 1
    print("*"*30)
print("="*70)
print("Final CNN test accuracy result each cross validation")
print(looop)
print("LOOCV Test accuracy mean in CNN: ", np.mean(looop))

0 Train accuracy: 0.759091 Test accuracy: 0.708333
1 Train accuracy: 0.820455 Test accuracy: 0.808333
2 Train accuracy: 0.915909 Test accuracy: 0.825
3 Train accuracy: 0.976515 Test accuracy: 0.925
4 Train accuracy: 0.989394 Test accuracy: 0.933333
5 Train accuracy: 0.990909 Test accuracy: 0.941667
6 Train accuracy: 0.996212 Test accuracy: 0.933333
7 Train accuracy: 0.998485 Test accuracy: 0.925
8 Train accuracy: 0.997727 Test accuracy: 0.941667
9 Train accuracy: 1.0 Test accuracy: 0.941667
10 Train accuracy: 1.0 Test accuracy: 0.933333
11 Train accuracy: 1.0 Test accuracy: 0.941667
12 Train accuracy: 1.0 Test accuracy: 0.941667
13 Train accuracy: 1.0 Test accuracy: 0.941667
14 Train accuracy: 1.0 Test accuracy: 0.933333
15 Train accuracy: 1.0 Test accuracy: 0.933333
16 Train accuracy: 1.0 Test accuracy: 0.933333
17 Train accuracy: 1.0 Test accuracy: 0.933333
18 Train accuracy: 1.0 Test accuracy: 0.933333
19 Train accuracy: 1.0 Test accuracy: 0.933333
1  times cross validation End
****

### Convolutional Neural Network Results Summary

1. CNN results way is same with DNN part
2. Final test set accuracy is 0.968056  // error rate = 1 - 0.968056 = 0.031944
3. It means 120 smaple classification is correct about 116 sample
4. fMRI data , DNN vs CNN in my case, CNN is better performing